In [1]:
import requests, re, math
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import json
import os
import mysql.connector
from mysql.connector import errorcode

# Web-scraping: Which sellers advertise/sponsor on eBay

### Search for buy-it-now listings of keyword and limit the number of items to 100 per page. For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt" and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code.

In [5]:
sponsored_links = list()
nonsponsored_links = list()

for i in range(1,11):
    
    #Take the url of ith search page for "playstation 4 slim".
    url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=playstation+4+slim&_sacat=0&_ipg=100&_pgn=%d'%i
    
    #Extract the url to a soup object.
    ps4 = get_html(url)
    
    #Transfer the html to BeautifulSoup object:
    searchPage = BeautifulSoup(ps4)
    items = searchPage.select("li.s-item")
    
    for item in items:
        if item.find('div', "s-item__title--tagblock")==None:
            link = re.findall(r'<a.+?class="s-item__link".+?href="(https:.+?)">',str(item))
            nonsponsored_links.append(link)
        else:  
            element = item.find('div', "s-item__title--tagblock").text
            sponsor = re.findall(r'.*S.*P.*O.*N.*S.*O.*R.*E.*D.*', element)

            if sponsor != []:
                link = re.findall(r'<a.+?class="s-item__link".+?href="(https:.+?)">',str(item))
                sponsored_links.append(link)

            else:
                link = re.findall(r'<a.+?class="s-item__link".+?href="(https:.+?)">',str(item))
                nonsponsored_links.append(link)
            
    #Generate 10s before opening next page:
    time.sleep(10)


In [6]:
sponsored_links = [x for x in sponsored_links if x != []]

In [7]:
# Write links to corresponding file:
with open("sponsored.txt", "w",encoding='utf-8') as file:
    for link in sponsored_links:
        file.write(link[0]+"\n")
        
with open("non-sponsored.txt", "w",encoding='utf-8') as file:
    for link in nonsponsored_links:
        file.write(link[0]+"\n")

### Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files and downloads each of the pages (URLs) into the folders "sponsored" and "non- sponsored". Each file should be named as "item-id.htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay.

In [9]:
# Create two folders:
path = os.getcwd()
sp_dir = os.path.join(path,'sponsored')
nonsp_dir = os.path.join(path,'non-sponsored')
os.mkdir(sp_dir)
os.mkdir(nonsp_dir)

In [10]:
# Read the file content
sp_data = open('sponsored.txt','r',encoding='utf-8')
sp_string = sp_data.read()
#close the file
sp_data.close()

nonsp_data = open('non-sponsored.txt','r',encoding='utf-8')
nonsp_string = nonsp_data.read()
#close the file
nonsp_data.close()

In [11]:
# Get the links of items from the string stored in files, and strap last element
# that contains nothing.
sp_links = sp_string.split('\n')
sp_links = sp_links[0:len(sp_links)-1]
nonsp_links = nonsp_string.split('\n')
nonsp_links = nonsp_links[0:len(nonsp_links)-1]

In [12]:
# Downloads each of the pages (URLs) into the folders "sponsored"
for j in range(len(sp_links)):
    try:
        #Extract the url to a soup object.
        page = get_html(sp_links[j])
    
        #Transfer the html to BeautifulSoup object:
        soup = BeautifulSoup(page)
        
        #Find the item id:
        item_id = soup.find('div',{'id':'descItemNumber'}).get_text()
        
        #Save content into file with corresponding name
        with open("sponsored/%s.htm"%item_id, "w",encoding='utf-8') as file:
            file.write(page)
            
        #Pause 2s between queries
        time.sleep(2)
        
    except: # Error handling
        pass
        print("Pass %dth sponsored item."%(j+1))

In [13]:
# Downloads each of the pages (URLs) into the folders "non-sponsored"
for k in range(len(nonsp_links)):
    try:
        #Extract the url to a soup object.
        page = get_html(nonsp_links[k])
    
        #Transfer the html to BeautifulSoup object:
        soup = BeautifulSoup(page)
        
        #Find the item id:
        item_id = soup.find('div',{'id':'descItemNumber'}).get_text()
        
        #Save content into file with corresponding name
        with open("non-sponsored/%s.htm"%item_id, "w",encoding='utf-8') as file:
            file.write(page)
            
        #Pause 2s between queries
        time.sleep(2)
        
    except: # Error handling
        pass
        print("Pass %dth non-sponsored item."%(k+1))

Pass 90th non-sponsored item.
Pass 426th non-sponsored item.


### Loops through the pages you downloaded and opens and parses them into a Python Beautifulsoup-object. Identify and select: seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...)

In [ ]:
item_id_list = list()
seller_name_list = list()
seller_score_list = list()
item_price_list = list()
num_item_sold_list = list()
best_offer_list = list()
returns_allowed_list = list()
shipping_price_list = list()
condition_list = list()
source_list = list()

for filename in os.listdir('sponsored/'):
    if filename != '.DS_Store':
        print("Getting info of %s"%filename)
        item_id_list.append(filename[0:12])
        item_data = open('sponsored/%s'%filename,'r',encoding='utf-8')
        content = item_data.read()
        item_data.close() #close the file
        
        #Transfer the html to BeautifulSoup object:
        item_soup = BeautifulSoup(content)

        # -------------------------------Find the seller name------------------------------------
        try:
            seller_name = item_soup.find('div',{'id':'RightSummaryPanel'}).find('span',{'class':'mbg-nw'}).get_text()
        except:
            seller_name = None
        seller_name_list.append(seller_name)

        # -------------------------------Find the seller score------------------------------------
        try:
            summary = item_soup.find('div',{'id':'RightSummaryPanel'})
            seller_score = re.findall(r'href="http://feedback.ebay.com.+>(.+)<', str(summary))[0]
        except:
            seller_score = None 
        seller_score_list.append(seller_score)

        # -------------------------------Find the item price------------------------------------
        try:
            item_price = item_soup.find('span',{'class':'notranslate','id':'prcIsum'}).get_text()
        except:
            try:
                item_price = item_soup.find('span',{'class':'notranslate','itemprop':'price'}).get_text()
            except:
                item_price = None 
        item_price_list.append(item_price)

        # -------------------------------Find the #item sold------------------------------------
        try:
            summary = item_soup.find('div',{'id':'mainContent'})
            num_item_sold = summary.find('a',{'class':'vi-txt-underline'}).get_text().split(' ')[0]
        except:
            num_item_sold = None
        num_item_sold_list.append(num_item_sold)

        # -------------------------------best offer available------------------------------------
        try:
            summary = item_soup.find('div',{'id':'mainContent'})
            best_offer = summary.find('div',{'class':'vi-bbox-dspn u-flL lable boLable'}).get_text()
            if len(best_offer)==11:
                best_offer = "Yes"
            else:
                best_offer = "No"
        except:
            best_offer = "No"
        best_offer_list.append(best_offer)
        
        # ---------------------------------returns allowed---------------------------------------
        try:
            returns_allowed = item_soup.find('span',{'id':'vi-ret-accrd-txt'}).get_text()
        except:
            returns_allowed = None
        returns_allowed_list.append(returns_allowed)

        # ----------------------------------shipping price---------------------------------------
        try:
            shipping_price = item_soup.find('span',{'id':'fshippingCost'}).get_text()
        except:
            try:
                shipping_price  = item_soup.find('span',{'id':'shSummary'}).find('strong',{'class':'sh_gr_bld'}).get_text()
            except:
                shipping_price = None
        shipping_price_list.append(shipping_price)

        # --------------------------------------condition-----------------------------------------
        try:
            condition = item_soup.find('div',{'itemprop':'itemCondition'}).get_text()
        except:
            condition = None
        condition_list.append(condition)
        
        source_list.append('sponsored')        
        

In [ ]:
for filename in os.listdir('non-sponsored/'):
    if filename != '.DS_Store':
        print("Getting info of %s"%filename)
        item_id_list.append(filename[0:12])
        item_data = open('non-sponsored/%s'%filename,'r',encoding='utf-8')
        content = item_data.read()
        item_data.close() #close the file
        
        #Transfer the html to BeautifulSoup object:
        item_soup = BeautifulSoup(content)

        # -------------------------------Find the seller name------------------------------------
        try:
            seller_name = item_soup.find('div',{'id':'RightSummaryPanel'}).find('span',{'class':'mbg-nw'}).get_text()
        except:
            seller_name = None
        seller_name_list.append(seller_name)

        # -------------------------------Find the seller score------------------------------------
        try:
            summary = item_soup.find('div',{'id':'RightSummaryPanel'})
            seller_score = re.findall(r'href="http://feedback.ebay.com.+>(.+)<', str(summary))[0]
        except:
            seller_score = None 
        seller_score_list.append(seller_score)

        # -------------------------------Find the item price------------------------------------
        try:
            item_price = item_soup.find('span',{'class':'notranslate','id':'prcIsum'}).get_text()
        except:
            try:
                item_price = item_soup.find('span',{'class':'notranslate','itemprop':'price'}).get_text()
            except:
                item_price = None 
        item_price_list.append(item_price)

        # -------------------------------Find the #item sold------------------------------------
        try:
            summary = item_soup.find('div',{'id':'mainContent'})
            num_item_sold = summary.find('a',{'class':'vi-txt-underline'}).get_text().split(' ')[0]
        except:
            num_item_sold = None
        num_item_sold_list.append(num_item_sold)

        # -------------------------------best offer available------------------------------------
        try:
            summary = item_soup.find('div',{'id':'mainContent'})
            best_offer = summary.find('div',{'class':'vi-bbox-dspn u-flL lable boLable'}).get_text()
            if len(best_offer)==11:
                best_offer = "Yes"
            else:
                best_offer = "No"
        except:
            best_offer = "No"
        best_offer_list.append(best_offer)
        
        # ---------------------------------returns allowed---------------------------------------
        try:
            returns_allowed = item_soup.find('span',{'id':'vi-ret-accrd-txt'}).get_text()
        except:
            returns_allowed = None
        returns_allowed_list.append(returns_allowed)

        # ----------------------------------shipping price---------------------------------------
        try:
            shipping_price = item_soup.find('span',{'id':'fshippingCost'}).get_text()
        except:
            try:
                shipping_price  = item_soup.find('span',{'id':'shSummary'}).find('strong',{'class':'sh_gr_bld'}).get_text()
            except:
                shipping_price = None
        shipping_price_list.append(shipping_price)

        # --------------------------------------condition-----------------------------------------
        try:
            condition = item_soup.find('div',{'itemprop':'itemCondition'}).get_text()
        except:
            condition = None
        condition_list.append(condition) 
        
        source_list.append('non-sponsored')        
   

In [6]:
# Transform the shipping price information to the right form.
shipping_price_list_test = shipping_price_list.copy()
for i in range(len(shipping_price_list)):
    if shipping_price_list[i] == '\nFREE\n':
        shipping_price_list_test[i] = 0
    elif shipping_price_list[i] == "FAST 'N FREE":
        shipping_price_list_test[i] = 0
    elif shipping_price_list[i] == None:
        shipping_price_list_test[i] = None
    else:
        prices = re.findall(r'[$](.*[.][0-9][0-9])',str(shipping_price_list[i]))
        if len(prices)==2:
            shipping_price_list_test[i] = int(re.sub("[^0-9]", "",prices[1]))
        else:
            shipping_price_list_test[i] = int(re.sub("[^0-9]", "",prices[0]))
    
shipping_price_list_formatted = shipping_price_list_test.copy()

In [7]:
# Transform the item price information to the right form.
item_price_list_test = item_price_list.copy()
currency_list = item_price_list.copy()
for i in range(len(item_price_list_test)):
    if item_price_list_test[i] == None:
        pass
    else: 
        if 'GBP' in item_price_list_test[i]:
            price = int(re.sub("[^0-9]", "", str(item_price_list_test[i])))
            item_price_list_test[i] = round(price*1.29) #1 GBP = 1.29 USD
            currency_list[i] = 'GBP'
        elif 'C' in item_price_list_test[i]:
            price = int(re.sub("[^0-9]", "", str(item_price_list_test[i])))
            item_price_list_test[i] = round(price*0.75)# 1 CAD = 0.75 USD
            currency_list[i] = 'Canadian Dollars'
        else:
            price = int(re.sub("[^0-9]", "", str(item_price_list_test[i])))
            item_price_list_test[i] = price
            currency_list[i] = 'USD'
            
item_price_list_formatted = item_price_list_test.copy()
currency_list_formatted = currency_list.copy()

In [8]:
# Transform the return information to the right form.
returns_allowed_list_test = returns_allowed_list.copy()
for i in range(len(returns_allowed_list_test)):
    if returns_allowed_list_test[i] == None:
        pass
    elif 'not' in returns_allowed_list_test[i]:
        returns_allowed_list_test[i] = 'No'
    else:
        returns_allowed_list_test[i] = 'Yes'
returns_allowed_list_formatted = returns_allowed_list_test.copy()


In [9]:
# The shipping cost note:
shipping_compute = [None]*len(shipping_price_list_formatted)
for i in range(len(shipping_price_list_formatted)):
    if shipping_price_list_formatted[i]==None:
        shipping_compute[i] = True
    else:
        shipping_compute[i] = False
shipping_cost_note_formatted = shipping_compute.copy()

### Connect to SQL. Create a database and name it "eBay". Save the information of items in into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored. If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert 12.34 into 1234 and 12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [11]:
# Connect to the database.
conn = mysql.connector.connect(
    host='localhost',
    user='root'
)

# Create a cursor.
cursor = conn.cursor()

# Create 'ucdavis' database.
cursor.execute("DROP DATABASE IF EXISTS eBay")
cursor.execute("CREATE DATABASE eBay")

# Check whether the database has be successfully created.
cursor.execute("SHOW DATABASES")
for x in cursor:
  print(x)

('eBay',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('ucdavis',)


In [12]:
# Use the 'ucdavis' database.
cursor.execute("USE eBay")

# Create table 'omdb' with corresponding columns.
cursor.execute("CREATE TABLE eBay_items (item_id CHAR(255), seller_name VARCHAR(255), seller_score INT(255), item_priceUSD INT(100), item_OriginalCurrency VARCHAR(255), num_item_sold INT(200), best_offer VARCHAR(255), returns_allowed VARCHAR(255), shipping_priceUSD INT(100), shipping_compute VARCHAR(255), conditions VARCHAR(255), source VARCHAR(255))")

# Check whether the table has be successfully created.
cursor.execute("SHOW TABLES")
for x in cursor:
  print(x)

('eBay_items',)


In [ ]:
# For each item, insert its information to the "eBay_items table."
sql = "INSERT INTO eBay_items (item_id, seller_name, seller_score, item_priceUSD, item_OriginalCurrency, num_item_sold, best_offer, returns_allowed, shipping_priceUSD, shipping_compute, conditions, source) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
for i in range(len(item_id_list)):
    item_id = item_id_list[i]
    seller_name = seller_name_list[i]
    seller_score = seller_score_list[i]
    item_price = item_price_list_formatted[i]
    item_currency = currency_list_formatted[i]
    num_item_sold = num_item_sold_list[i]
    shipping_price = shipping_price_list_formatted[i]
    shipping_compute = shipping_cost_note_formatted[i]
    best_offer = best_offer_list[i]
    returns_allowed = returns_allowed_list_formatted[i]
    condition = condition_list[i]
    source = source_list[i]
    
    val = (item_id, seller_name, seller_score, item_price, item_currency, num_item_sold, 
           best_offer, returns_allowed, shipping_price, shipping_compute,
           condition, source)
    cursor.execute("set sql_mode = ''")
    
    cursor.execute(sql, val)

# Make the changes to the database.
conn.commit()

# Check whether the information has been successfully inserted.
cursor.execute("SELECT * FROM eBay_items")
result = cursor.fetchall()
for x in result:
  print(x)

### Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations. 

In [ ]:
# Categorical: item_id, seller_name, conditions, source, item_currency
# Binary categorical: best_offer, shipping_compute, returns_allowed
# numeric: seller_score, item_price, shipping_price, num_item_sold

cursor.execute("SELECT source, item_id, count(item_id) FROM eBay_items GROUP BY source, item_id")
result = cursor.fetchall()
print("-----------------------------------Item ID-----------------------------------------")
print("Source           item_id    Count")
for x in result:
    print(x)

In [ ]:
cursor.execute("SELECT source, seller_name, count(seller_name) FROM eBay_items GROUP BY source, seller_name")
result = cursor.fetchall()
print("-----------------------------------Seller Name-----------------------------------------")
print("Source           seller_name         Count")
for x in result:
    print(x)  

In [16]:
cursor.execute("SELECT source, item_OriginalCurrency, count(item_OriginalCurrency) FROM eBay_items GROUP BY source, item_OriginalCurrency")
result = cursor.fetchall()
print("-------------------------------Item Original Currency-----------------------------------")
print("Source      Currency  Count")
for x in result:
    print(x)

-------------------------------Item Original Currency-----------------------------------
Source      Currency  Count
('sponsored', 'USD', 94)
('sponsored', 'Canadian Dollars', 8)
('non-sponsored', 'USD', 806)


In [17]:
cursor.execute("SELECT source, conditions, avg(returns_allowed_binary), min(returns_allowed_binary), max(returns_allowed_binary), std(returns_allowed_binary) FROM (SELECT source, conditions, (CASE WHEN returns_allowed='Yes' THEN 1 WHEN returns_allowed='No' THEN 0 ELSE Null END) AS returns_allowed_binary FROM eBay_items) AS a GROUP BY source, conditions")
result = cursor.fetchall()
print("---------------------------------Returns Allowed---------------------------------------")
print("Source       Condition         Mean    Min    Max  Standard Deviation")
for x in result:
    print(x)

---------------------------------Returns Allowed---------------------------------------
Source       Condition         Mean    Min    Max  Standard Deviation
('sponsored', 'Used', Decimal('0.5385'), 0, 1, 0.4985185152621431)
('sponsored', 'New', Decimal('0.7143'), 0, 1, 0.45175395145262554)
('sponsored', 'Open box', Decimal('0.5000'), 0, 1, 0.5)
('sponsored', 'Seller refurbished', Decimal('1.0000'), 1, 1, 0.0)
('sponsored', 'Manufacturer refurbished', Decimal('1.0000'), 1, 1, 0.0)
('non-sponsored', 'Used', Decimal('0.4391'), 0, 1, 0.496274933497733)
('non-sponsored', 'Seller refurbished', Decimal('0.5357'), 0, 1, 0.4987228587060336)
('non-sponsored', 'New', Decimal('0.5106'), 0, 1, 0.49988681380726496)
('non-sponsored', 'Manufacturer refurbished', Decimal('0.2857'), 0, 1, 0.45175395145262565)
('non-sponsored', 'Open box', Decimal('0.2075'), 0, 1, 0.4055506652868808)


In [18]:
cursor.execute("SELECT source, conditions, avg(shipping_compute_binary), min(shipping_compute_binary), max(shipping_compute_binary), std(shipping_compute_binary) FROM (SELECT source, conditions, (CASE WHEN shipping_compute=1 THEN 1 ELSE 0 END) AS shipping_compute_binary FROM eBay_items) AS a GROUP BY source, conditions")
result = cursor.fetchall()
print("---------------------------------Shipping Compute---------------------------------------")
print("Source       Condition         Mean    Min    Max  Standard Deviation")
for x in result:
    print(x)

---------------------------------Shipping Compute---------------------------------------
Source       Condition         Mean    Min    Max  Standard Deviation
('sponsored', 'Used', Decimal('0.3654'), 0, 1, 0.4815378471364968)
('sponsored', 'New', Decimal('0.2857'), 0, 1, 0.45175395145262554)
('sponsored', 'Open box', Decimal('0.5000'), 0, 1, 0.5)
('sponsored', 'Seller refurbished', Decimal('0.0000'), 0, 0, 0.0)
('sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0, 0.0)
('non-sponsored', 'Used', Decimal('0.3876'), 0, 1, 0.4872058493091433)
('non-sponsored', 'Seller refurbished', Decimal('0.3929'), 0, 1, 0.48838551182776224)
('non-sponsored', 'New', Decimal('0.2411'), 0, 1, 0.4277718822695097)
('non-sponsored', 'Manufacturer refurbished', Decimal('0.7143'), 0, 1, 0.45175395145262565)
('non-sponsored', 'Open box', Decimal('0.6038'), 0, 1, 0.4891125055402158)


In [19]:
cursor.execute("SELECT source, conditions, avg(best_offer_binary), min(best_offer_binary), max(best_offer_binary), std(best_offer_binary) FROM (SELECT source, conditions, (CASE WHEN best_offer='Yes' THEN 1 ELSE 0 END) AS best_offer_binary FROM eBay_items) AS a GROUP BY source, conditions")
result = cursor.fetchall()
print("-------------------------------Best Offer Available-------------------------------------")
print("Source       Condition         Mean    Min    Max  Standard Deviation")
for x in result:
    print(x) 

-------------------------------Best Offer Available-------------------------------------
Source       Condition         Mean    Min    Max  Standard Deviation
('sponsored', 'Used', Decimal('0.5385'), 0, 1, 0.498518515262143)
('sponsored', 'New', Decimal('0.2619'), 0, 1, 0.43967107887189016)
('sponsored', 'Open box', Decimal('0.7500'), 0, 1, 0.4330127018922193)
('sponsored', 'Seller refurbished', Decimal('0.0000'), 0, 0, 0.0)
('sponsored', 'Manufacturer refurbished', Decimal('1.0000'), 1, 1, 0.0)
('non-sponsored', 'Used', Decimal('0.4083'), 0, 1, 0.49151113931154633)
('non-sponsored', 'Seller refurbished', Decimal('0.4286'), 0, 1, 0.4948716593053935)
('non-sponsored', 'New', Decimal('0.2872'), 0, 1, 0.4524717088965263)
('non-sponsored', 'Manufacturer refurbished', Decimal('0.5714'), 0, 1, 0.4948716593053935)
('non-sponsored', 'Open box', Decimal('0.5283'), 0, 1, 0.49919836057822464)


In [20]:
cursor.execute("SELECT source, conditions, avg(seller_score), min(seller_score), max(seller_score), std(seller_score) FROM eBay_items GROUP BY source, conditions")
result = cursor.fetchall()
print("-----------------------------------Seller Score-----------------------------------------")
print("Source       Condition         Mean    Min    Max  Standard Deviation")
for x in result:
    print(x)

-----------------------------------Seller Score-----------------------------------------
Source       Condition         Mean    Min    Max  Standard Deviation
('sponsored', 'Used', Decimal('19418.5769'), 0, 258907, 47425.028035086056)
('sponsored', 'New', Decimal('16724.8571'), 0, 568381, 86473.11589890413)
('sponsored', 'Open box', Decimal('16498.7500'), 5, 64483, 27710.06277848356)
('sponsored', 'Seller refurbished', Decimal('20130.6667'), 325, 41454, 16825.10315636199)
('sponsored', 'Manufacturer refurbished', Decimal('534.0000'), 534, 534, 0.0)
('non-sponsored', 'Used', Decimal('39041.7862'), 0, 258909, 67882.4444101479)
('non-sponsored', 'Seller refurbished', Decimal('99119.8571'), 0, 2265576, 420175.51862998825)
('non-sponsored', 'New', Decimal('15079.0709'), -1, 916859, 65110.53533610238)
('non-sponsored', 'Manufacturer refurbished', Decimal('323663.0000'), 0, 2265577, 792783.0708413061)
('non-sponsored', 'Open box', Decimal('24560.8679'), 0, 620455, 113206.78483984523)


In [21]:
cursor.execute("SELECT source, conditions, avg(shipping_priceUSD), min(shipping_priceUSD), max(shipping_priceUSD), std(shipping_priceUSD) FROM eBay_items GROUP BY source, conditions")
result = cursor.fetchall()
print("-------------------------------Shipping Price USD---------------------------------------")
print("Source       Condition         Mean    Min    Max  Standard Deviation")
for x in result:
    print(x)

-------------------------------Shipping Price USD---------------------------------------
Source       Condition         Mean    Min    Max  Standard Deviation
('sponsored', 'Used', Decimal('1291.3636'), 0, 5266, 1993.5356867724392)
('sponsored', 'New', Decimal('339.9333'), 0, 4999, 1184.1024993170518)
('sponsored', 'Open box', Decimal('0.0000'), 0, 0, 0.0)
('sponsored', 'Seller refurbished', Decimal('0.0000'), 0, 0, 0.0)
('sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0, 0.0)
('non-sponsored', 'Used', Decimal('542.7678'), 0, 4660, 943.4716424387796)
('non-sponsored', 'Seller refurbished', Decimal('216.3529'), 0, 2000, 595.0747794013662)
('non-sponsored', 'New', Decimal('303.6308'), 0, 4800, 752.8772902598483)
('non-sponsored', 'Manufacturer refurbished', Decimal('75.0000'), 0, 150, 75.0)
('non-sponsored', 'Open box', Decimal('926.4286'), 0, 5700, 1393.4997589634856)


In [22]:
cursor.execute("SELECT source, conditions, avg(item_priceUSD), min(item_priceUSD), max(item_priceUSD), std(item_priceUSD) FROM eBay_items GROUP BY source, conditions")
result = cursor.fetchall()
print("----------------------------------Item Price USD---------------------------------------")
print("Source       Condition         Mean    Min    Max  Standard Deviation")
for x in result:
    print(x)

----------------------------------Item Price USD---------------------------------------
Source       Condition         Mean    Min    Max  Standard Deviation
('sponsored', 'Used', Decimal('25742.8077'), 15000, 55351, 7721.590168627738)
('sponsored', 'New', Decimal('33389.9524'), 22100, 59999, 10738.533953660663)
('sponsored', 'Open box', Decimal('25073.7500'), 18995, 33500, 5261.375028212682)
('sponsored', 'Seller refurbished', Decimal('22232.6667'), 19999, 24700, 1926.2745275675418)
('sponsored', 'Manufacturer refurbished', Decimal('19999.0000'), 19999, 19999, 0.0)
('non-sponsored', 'Used', Decimal('21516.6445'), 1450, 50000, 6513.247823182019)
('non-sponsored', 'Seller refurbished', Decimal('20966.3214'), 7100, 30000, 5341.458555846571)
('non-sponsored', 'New', Decimal('30604.2092'), 4600, 157500, 10956.995966283972)
('non-sponsored', 'Manufacturer refurbished', Decimal('18292.7143'), 10250, 25000, 5182.301328678649)
('non-sponsored', 'Open box', Decimal('24747.3019'), 12400, 50000, 

In [23]:
cursor.execute("SELECT source, conditions, avg(num_item_sold), min(num_item_sold), max(num_item_sold), std(num_item_sold) FROM eBay_items GROUP BY source, conditions")
result = cursor.fetchall()
print("---------------------------------Num of Items Sold--------------------------------------")
print("Source       Condition         Mean    Min    Max  Standard Deviation")
for x in result:
    print(x)

---------------------------------Num of Items Sold--------------------------------------
Source       Condition         Mean    Min    Max  Standard Deviation
('sponsored', 'Used', Decimal('22.5000'), 3, 58, 21.523243250030884)
('sponsored', 'New', Decimal('39.4118'), 1, 316, 74.81196729018018)
('sponsored', 'Open box', Decimal('5.0000'), 5, 5, 0.0)
('sponsored', 'Seller refurbished', Decimal('2.5000'), 1, 4, 1.5)
('sponsored', 'Manufacturer refurbished', Decimal('3.0000'), 3, 3, 0.0)
('non-sponsored', 'Used', Decimal('11.1000'), 1, 58, 16.573774464496616)
('non-sponsored', 'Seller refurbished', Decimal('23.2857'), 1, 93, 33.37113502806317)
('non-sponsored', 'New', Decimal('37.5303'), 1, 872, 129.32074521256214)
('non-sponsored', 'Manufacturer refurbished', Decimal('5.0000'), 5, 5, 0.0)
('non-sponsored', 'Open box', Decimal('41.0000'), 1, 119, 55.16037224916694)


In [24]:
cursor.execute("SELECT source, conditions, count(conditions) FROM eBay_items GROUP BY source, conditions")
result = cursor.fetchall()
print("-------------------------------------Condition-----------------------------------------")
print("Source           conditions         Count")
for x in result:
    print(x) 

-------------------------------------Condition-----------------------------------------
Source           conditions         Count
('sponsored', 'Used', 52)
('sponsored', 'New', 42)
('sponsored', 'Open box', 4)
('sponsored', 'Seller refurbished', 3)
('sponsored', 'Manufacturer refurbished', 1)
('non-sponsored', 'Used', 436)
('non-sponsored', 'Seller refurbished', 28)
('non-sponsored', 'New', 282)
('non-sponsored', 'Manufacturer refurbished', 7)
('non-sponsored', 'Open box', 53)


### Use the stats in above and tell how sponsored and non-sponsored items appear to be different. 

For seller name, shipping price, best offer available, and conditions, there is no much difference between the sponsor and non-sponsor items. 

Number of item sold: the non-sponsored new item have a larger variance of number of items sold. The maximum is a lot greater than that of sponsored new item. Overall, the sponsored items that are new or used are sold the most while the non-sponsored items that are new or seller refurbished are sold the most.

Item price: all sponsored items have higher min, average, and max item prices than 
non-sponsored items.

Shipping price: all sponsored items have higher average shipping prices than non-sponsored items.

Seller score: except the items that are new, all sponsored items have  higher mean and max seller scores than that of non-sponsored items.

Returns allowed: overall, the sponsored items have higher chance of accepting returns than that of non-sponsored items.

Best offer available: from the mean, overall, the sponsored items have higher chance of best offer available than that of non-sponsored items.

In a nutshell, columns "number of item sold", "item price", "seller score", and "returns allowed" can be used to predict the sponsor/non-sponsor items.